### TMA4320 - Project 2: Physics


# 0.0 Motivation

In [11]:
print("this is the second commit on the laptop-branch, from the large computer")



In [9]:
# Importing libraries
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [5]:
# Parameters
sigma_0 = 1000 # kg/m^2
sigma = 500 # kg/m^2
R = 10 # m, Radius
A_s = 0.5*np.pi*(R**2) # Cross-section of boat
g = 9.81 # m/s^2

NameError: name 'np' is not defined

# X.0 Conclusion

Lorem Ipsum...